In [1]:
import numpy as np
import pandas as pd
import ast
import tensorflow as tf

This is just a script of model training and the codes are repetition from NCF_model hence we will not make any repeated annotation.

Load train data which is 80% of the entire playlist and the last column is list of 49 tracks of 0 label tracks. 

Load test data which is 20% of playlist without negative data

In [2]:
train_data_with_label = pd.read_csv("trainDF61.csv", sep="\t", encoding="utf-8")

In [3]:
train_data_with_label.head()

,pid,track_id,label
0,124726,19717,0
1,142752,422369,0
2,121208,238275,1
3,123947,391649,1
4,114258,335513,0


In [4]:
test_DF = pd.read_csv("testWithoutNegative.csv", sep='\t',  encoding='utf-8', index_col =None)
test_DF = test_DF.loc[:,['pid', 'encoded_track_uri']]
test_DF.head()

,pid,encoded_track_uri
0,102489,"[215623, 393454, 69228, 4854, 439226, 165886, ..."
1,10787,"[154957, 453219, 104309, 389316, 37543, 158183..."
2,135699,"[263624, 78766, 68030, 263570, 266054, 190141,..."
3,105545,"[323172, 426350, 386201, 343344, 294639, 31349..."
4,100888,"[128887, 75980, 51200, 368949, 233305, 438419,..."


In [5]:
test_DF = test_DF.loc[:,['pid','encoded_track_uri']].reset_index(drop=True).rename(columns={'encoded_track_uri': 'track_id'})
test_DF['track_id'] = test_DF['track_id'].apply(ast.literal_eval)
test_DF = test_DF.explode("track_id")
test_DF['label'] = 1
test_DF = test_DF.sample(frac=1, random_state=1)
test_DF.head()

,pid,track_id,label
3938,102312,310283,1
2298,14157,26349,1
1036,113235,312440,1
3630,129834,20220,1
1183,124700,16483,1


In [6]:
user_train = np.array(train_data_with_label['pid'], dtype=int)
item_train = np.array(train_data_with_label['track_id'], dtype=int)
label_train = np.array(train_data_with_label['label'], dtype=int)

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices(({"user_input": user_train, "item_input": item_train}, label_train))

2023-04-24 13:42:58.908544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 13:42:59.013497: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 13:42:59.015199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 13:42:59.021464: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
test_user = tf.convert_to_tensor(np.array(test_DF['pid'], dtype=int), dtype=tf.int32)
test_item = tf.convert_to_tensor(np.array(test_DF['track_id'], dtype=int), dtype=tf.int32)
test_tensor = {"user_input": test_user, "item_input": test_item}

In [15]:
batch_size = 2048
train_data = tf.data.Dataset.from_tensor_slices(((user_train, item_train), label_train))
train_data = train_data.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
num_users = train_data_with_label.loc[train_data_with_label['label'] == 1, 'pid'].nunique()
num_items = train_data_with_label.loc[train_data_with_label['label'] == 1, 'track_id'].nunique()

In [17]:
def get_model(num_users, num_items, mf_dim=8, layers=[64, 32, 16, 8], reg_layers=[0, 0, 0, 0], reg_mf = [0,0]):
    
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = tf.keras.Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = tf.keras.Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layer
    MF_Embedding_User = tf.keras.layers.Embedding(input_dim = num_users, output_dim = mf_dim, name = 'mf_embedding_user',
                                  embeddings_initializer='normal', embeddings_regularizer = tf.keras.regularizers.l2(reg_mf[0]), input_length=1)
    MF_Embedding_Item = tf.keras.layers.Embedding(input_dim = num_items, output_dim = mf_dim, name = 'mf_embedding_item',
                                  embeddings_initializer='normal', embeddings_regularizer = tf.keras.regularizers.l2(reg_mf[1]), input_length=1)   

    MLP_Embedding_User = tf.keras.layers.Embedding(input_dim = num_users, output_dim = int(layers[0]/2), name = "mlp_embedding_user",
                                  embeddings_initializer='normal', embeddings_regularizer = tf.keras.regularizers.l2(reg_layers[0]), input_length=1)
    MLP_Embedding_Item = tf.keras.layers.Embedding(input_dim = num_items, output_dim = int(layers[0]/2), name = 'mlp_embedding_item',
                                  embeddings_initializer='normal', embeddings_regularizer = tf.keras.regularizers.l2(reg_layers[0]), input_length=1)   
    
    # MF part
    mf_user_latent = tf.keras.layers.Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = tf.keras.layers.Flatten()(MF_Embedding_Item(item_input))
    mf_vector = tf.keras.layers.Multiply()([mf_user_latent, mf_item_latent]) # element-wise multiply 

    # MLP part 
    mlp_user_latent = tf.keras.layers.Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = tf.keras.layers.Flatten()(MLP_Embedding_Item(item_input))
    mlp_vector = tf.keras.layers.Concatenate()([mlp_user_latent, mlp_item_latent])
    for idx in range(1, num_layer):
        layer = tf.keras.layers.Dense(layers[idx], kernel_regularizer= tf.keras.regularizers.l2(reg_layers[idx]), activation='relu', name="layer%d" %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    #mf_vector = Lambda(lambda x: x * alpha)(mf_vector)
    #mlp_vector = Lambda(lambda x : x * (1-alpha))(mlp_vector)
    predict_vector = tf.keras.layers.Concatenate()([mf_vector, mlp_vector])
    
    # Final prediction layer
    prediction = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)
    
    model = tf.keras.Model(inputs=[user_input, item_input], 
                  outputs=prediction)
    
    return model

In [91]:
model = get_model(num_users, num_items) #default papers model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [92]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
mlp_embedding_user (Embedding)  (None, 1, 32)        1189472     user_input[0][0]                 
__________________________________________________________________________________________________
mlp_embedding_item (Embedding)  (None, 1, 32)        13437376    item_input[0][0]                 
___________________________________________________________________________________________

In [93]:
%%time
history = model.fit(train_data, epochs=20) #vanilla

Epoch 1/20
4805/4805 [==============================] - 46s 9ms/step - loss: 0.4603 - accuracy: 0.7696
Epoch 2/20
4805/4805 [==============================] - 44s 9ms/step - loss: 0.3929 - accuracy: 0.8150
Epoch 3/20
4805/4805 [==============================] - 44s 9ms/step - loss: 0.2991 - accuracy: 0.8696
Epoch 4/20
4805/4805 [==============================] - 44s 9ms/step - loss: 0.2383 - accuracy: 0.8984
Epoch 5/20
4805/4805 [==============================] - 45s 9ms/step - loss: 0.2081 - accuracy: 0.9101
Epoch 6/20
4805/4805 [==============================] - 44s 9ms/step - loss: 0.1870 - accuracy: 0.9176
Epoch 7/20
4805/4805 [==============================] - 44s 9ms/step - loss: 0.1738 - accuracy: 0.9218
Epoch 8/20
4805/4805 [==============================] - 44s 9ms/step - loss: 0.1630 - accuracy: 0.9251
Epoch 9/20
4805/4805 [==============================] - 45s 9ms/step - loss: 0.1593 - accuracy: 0.9260
Epoch 10/20
4805/4805 [==============================] - 44s 9ms/step - l

In [94]:
score = model.predict(test_tensor)

In [95]:
import pickle
with open('/home/jupyter/model_results/latent_8.pkl', 'wb') as file:
    pickle.dump(history.history, file)

In [96]:
np.save('/home/jupyter/model_results/latent_8.npy', score)

In [100]:
results = {}
predictions = {}
latent_dimension = [16, 32, 64, 128, 256]

for i in latent_dimension:
    model = get_model(num_users, num_items, i)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    results["latent_" + str(i)] = model.fit(train_data, epochs=20)
    predictions["latent=" + str(i)] = model.predict(test_tensor) 

    spath = "/home/jupyter/model_results/" + "latent_" +str(i)+".pkl"
    with open(spath, 'wb') as file:
        pickle.dump(results["latent_" + str(i)].history, file)
    print(f"{spath} -- saved")
    
    path = "/home/jupyter/model_results/" + "latent_" +str(i) + ".npy"
    np.save(path, predictions["latent=" + str(i)])
    print(f"{path} -- saved")

Epoch 1/20
4805/4805 [==============================] - 52s 11ms/step - loss: 0.4623 - accuracy: 0.7676
Epoch 2/20
4805/4805 [==============================] - 52s 11ms/step - loss: 0.3964 - accuracy: 0.8139
Epoch 3/20
4805/4805 [==============================] - 51s 11ms/step - loss: 0.2998 - accuracy: 0.8682
Epoch 4/20
4805/4805 [==============================] - 51s 11ms/step - loss: 0.2328 - accuracy: 0.9009
Epoch 5/20
4805/4805 [==============================] - 51s 11ms/step - loss: 0.2003 - accuracy: 0.9140
Epoch 6/20
4805/4805 [==============================] - 52s 11ms/step - loss: 0.1840 - accuracy: 0.9199
Epoch 7/20
4805/4805 [==============================] - 51s 11ms/step - loss: 0.1699 - accuracy: 0.9237
Epoch 8/20
4805/4805 [==============================] - 51s 11ms/step - loss: 0.1616 - accuracy: 0.9258
Epoch 9/20
4805/4805 [==============================] - 51s 11ms/step - loss: 0.1573 - accuracy: 0.9263
Epoch 10/20
4805/4805 [==============================] - 51s 11m

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:442: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 107499008 elements. This may consume a large amount of memory.
  num_elements)


4805/4805 [==============================] - 259s 54ms/step - loss: 0.4622 - accuracy: 0.7665
Epoch 2/20
4805/4805 [==============================] - 258s 54ms/step - loss: 0.3809 - accuracy: 0.8216
Epoch 3/20
4805/4805 [==============================] - 258s 54ms/step - loss: 0.2832 - accuracy: 0.8750
Epoch 4/20
4805/4805 [==============================] - 258s 54ms/step - loss: 0.2245 - accuracy: 0.9025
Epoch 5/20
4805/4805 [==============================] - 258s 54ms/step - loss: 0.1978 - accuracy: 0.9133
Epoch 6/20
4805/4805 [==============================] - 258s 54ms/step - loss: 0.1823 - accuracy: 0.9189
Epoch 7/20
1919/4805 [==========>...................] - ETA: 2:35 - loss: 0.1731 - accuracy: 0.9217

KeyboardInterrupt: 

In [ ]:
# get_model(num_users, num_items, mf_dim=8, layers=[64, 32, 16, 8], reg_layers=[0, 0, 0, 0], reg_mf = [0,0])

In [101]:
%%time
MLP_Layers = [[16, 8], [32, 16], [64, 32] ,[32, 16, 8], [64, 32, 16], [128, 64, 32], [256, 128, 64, 32]]
REG_Layers = [[0, 0], [0, 0], [0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0]]

for i in range(len(MLP_Layers)):
    model = get_model(num_users, num_items, mf_dim=8, layers=MLP_Layers[i], reg_layers=REG_Layers[i], reg_mf = [0,0] )
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    results["MLP_" + str(i)] = model.fit(train_data, epochs=20)
    predictions["MLP=" + str(i)] = model.predict(test_tensor) 

    spath = "/home/jupyter/model_results/" + "MLP_" +str(i)+".pkl"
    with open(spath, 'wb') as file:
        pickle.dump(results["MLP_" + str(i)].history, file)
    print(f"{spath} -- saved")
    
    path = "/home/jupyter/model_results/" + "MLP_" +str(i) + ".npy"
    np.save(path, predictions["MLP=" + str(i)])
    print(f"{path} -- saved")

Epoch 1/20
4805/4805 [==============================] - 23s 5ms/step - loss: 0.4729 - accuracy: 0.7520
Epoch 2/20
4805/4805 [==============================] - 23s 5ms/step - loss: 0.4134 - accuracy: 0.7965
Epoch 3/20
4805/4805 [==============================] - 22s 5ms/step - loss: 0.3800 - accuracy: 0.8209
Epoch 4/20
4805/4805 [==============================] - 22s 5ms/step - loss: 0.3423 - accuracy: 0.8332
Epoch 5/20
4805/4805 [==============================] - 23s 5ms/step - loss: 0.3046 - accuracy: 0.8527
Epoch 6/20
4805/4805 [==============================] - 22s 5ms/step - loss: 0.2842 - accuracy: 0.8657
Epoch 7/20
4805/4805 [==============================] - 23s 5ms/step - loss: 0.2677 - accuracy: 0.8752
Epoch 8/20
4805/4805 [==============================] - 22s 5ms/step - loss: 0.2545 - accuracy: 0.8805
Epoch 9/20
4805/4805 [==============================] - 22s 5ms/step - loss: 0.2441 - accuracy: 0.8844
Epoch 10/20
4805/4805 [==============================] - 23s 5ms/step - l

In [102]:
latent_dimension = [16, 32, 64]
MLP_Layers = [[16, 8], [32, 16], [64, 32] ,[32, 16, 8], [64, 32, 16], [128, 64, 32], [256, 128, 64, 32]]
REG_Layers = [[0, 0], [0, 0], [0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0]]

for index, value in enumerate(MLP_Layers):
    for j in latent_dimension: 
        model = get_model(num_users, num_items, mf_dim=j, layers=value, reg_layers=REG_Layers[index], reg_mf = [0,0])
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
        results["MLP_" + str(i) + "_N_" + "lat_" + str(j)] = model.fit(train_data, epochs=20)
        predictions["MLP_" + str(i) + "_N_" + "lat_" + str(j)] = model.predict(test_tensor) 

        spath = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(index) + "_lat" + str(j) + ".pkl"
        with open(spath, 'wb') as file:
            pickle.dump(results["MLP_" + str(i) + "_N_" + "lat_" + str(j)].history, file)
        print(f"{spath} -- saved")

        path = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(index) + "_lat" + str(j) + ".npy"
        np.save(path, predictions["MLP_" + str(i) + "_N_" + "lat_" + str(j)])
        print(f"{path} -- saved")

Epoch 1/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.4731 - accuracy: 0.7520
Epoch 2/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.4152 - accuracy: 0.7967
Epoch 3/20
4805/4805 [==============================] - 29s 6ms/step - loss: 0.3793 - accuracy: 0.8188
Epoch 4/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.3395 - accuracy: 0.8328
Epoch 5/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.3027 - accuracy: 0.8551
Epoch 6/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.2786 - accuracy: 0.8685
Epoch 7/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.2603 - accuracy: 0.8762
Epoch 8/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.2476 - accuracy: 0.8819
Epoch 9/20
4805/4805 [==============================] - 30s 6ms/step - loss: 0.2373 - accuracy: 0.8875
Epoch 10/20
4805/4805 [==============================] - 30s 6ms/step - l

KeyboardInterrupt: 

In [15]:
latent_dimension = [16, 32, 64]
MLP_Layers = [[16, 8], [32, 16], [64, 32] ,[32, 16, 8], [64, 32, 16], [128, 64, 32], [256, 128, 64, 32]]
REG_Layers = [[0, 0], [0, 0], [0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0]]

for index, value in enumerate(MLP_Layers):
    if index == 6:
        for j in latent_dimension: 
            model = get_model(num_users, num_items, mf_dim=j, layers=value, reg_layers=REG_Layers[index], reg_mf = [0,0])
            model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
            results["MLP_" + str(i) + "_N_" + "lat_" + str(j)] = model.fit(train_data, epochs=20)
            predictions["MLP_" + str(i) + "_N_" + "lat_" + str(j)] = model.predict(test_tensor) 

            spath = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(index) + "_lat" + str(j) + ".pkl"
            with open(spath, 'wb') as file:
                pickle.dump(results["MLP_" + str(i) + "_N_" + "lat_" + str(j)].history, file)
            print(f"{spath} -- saved")

            path = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(index) + "_lat" + str(j) + ".npy"
            np.save(path, predictions["MLP_" + str(i) + "_N_" + "lat_" + str(j)])
            print(f"{path} -- saved")
    else:
        continue

Epoch 1/20
4805/4805 [==============================] - 137s 28ms/step - loss: 0.4547 - accuracy: 0.7770
Epoch 2/20
4805/4805 [==============================] - 135s 28ms/step - loss: 0.3312 - accuracy: 0.8558
Epoch 3/20
4805/4805 [==============================] - 135s 28ms/step - loss: 0.2036 - accuracy: 0.9167
Epoch 4/20
4805/4805 [==============================] - 135s 28ms/step - loss: 0.1603 - accuracy: 0.9320
Epoch 5/20
4805/4805 [==============================] - 135s 28ms/step - loss: 0.1391 - accuracy: 0.9381
Epoch 6/20
4805/4805 [==============================] - 137s 29ms/step - loss: 0.1271 - accuracy: 0.9413
Epoch 7/20
4805/4805 [==============================] - 138s 29ms/step - loss: 0.1191 - accuracy: 0.9436
Epoch 8/20
4805/4805 [==============================] - 137s 29ms/step - loss: 0.1144 - accuracy: 0.9449
Epoch 9/20
4805/4805 [==============================] - 137s 29ms/step - loss: 0.1123 - accuracy: 0.9456
Epoch 10/20
4805/4805 [==============================] 

NameError: name 'i' is not defined

In [18]:
results = {}
predictions = {}

In [19]:
import pickle 
import numpy as np

In [20]:
model = get_model(num_users, num_items, mf_dim=16, layers=[256, 128, 64, 32], reg_layers=[0,0,0,0], reg_mf = [0,0])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
results["MLP_" + str(6) + "_N_" + "lat_" + str(1)] = model.fit(train_data, epochs=20)
predictions["MLP_" + str(6) + "_N_" + "lat_" + str(1)] = model.predict(test_tensor) 

spath = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(6) + "_lat" + str(1) + ".pkl"
with open(spath, 'wb') as file:
    pickle.dump(results["MLP_" + str(6) + "_N_" + "lat_" + str(1)].history, file)
print(f"{spath} -- saved")

path = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(6) + "_lat" + str(1) + ".npy"
np.save(path, predictions["MLP_" + str(6) + "_N_" + "lat_" + str(1)])
print(f"{path} -- saved")

Epoch 1/20
2403/2403 [==============================] - 70s 29ms/step - loss: 0.4591 - accuracy: 0.7723
Epoch 2/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.3617 - accuracy: 0.8361
Epoch 3/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.2261 - accuracy: 0.9045
Epoch 4/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.1796 - accuracy: 0.9210
Epoch 5/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.1599 - accuracy: 0.9270
Epoch 6/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.1489 - accuracy: 0.9302
Epoch 7/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.1436 - accuracy: 0.9311
Epoch 8/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.1398 - accuracy: 0.9322
Epoch 9/20
2403/2403 [==============================] - 69s 29ms/step - loss: 0.1397 - accuracy: 0.9319
Epoch 10/20
2403/2403 [==============================] - 69s 29m

In [21]:
model = get_model(num_users, num_items, mf_dim=32, layers=[256, 128, 64, 32], reg_layers=[0,0,0,0], reg_mf = [0,0])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
results["MLP_" + str(6) + "_N_" + "lat_" + str(2)] = model.fit(train_data, epochs=20)
predictions["MLP_" + str(6) + "_N_" + "lat_" + str(2)] = model.predict(test_tensor) 

spath = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(6) + "_lat" + str(2) + ".pkl"
with open(spath, 'wb') as file:
    pickle.dump(results["MLP_" + str(6) + "_N_" + "lat_" + str(2)].history, file)
print(f"{spath} -- saved")

path = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(6) + "_lat" + str(2) + ".npy"
np.save(path, predictions["MLP_" + str(6) + "_N_" + "lat_" + str(2)])
print(f"{path} -- saved")

Epoch 1/20
2403/2403 [==============================] - 77s 31ms/step - loss: 0.4591 - accuracy: 0.7727
Epoch 2/20
2403/2403 [==============================] - 76s 32ms/step - loss: 0.3566 - accuracy: 0.8395
Epoch 3/20
2403/2403 [==============================] - 76s 31ms/step - loss: 0.2195 - accuracy: 0.9078
Epoch 4/20
2403/2403 [==============================] - 76s 32ms/step - loss: 0.1748 - accuracy: 0.9233
Epoch 5/20
2403/2403 [==============================] - 76s 31ms/step - loss: 0.1564 - accuracy: 0.9283
Epoch 6/20
2403/2403 [==============================] - 76s 31ms/step - loss: 0.1453 - accuracy: 0.9310
Epoch 7/20
2403/2403 [==============================] - 76s 31ms/step - loss: 0.1400 - accuracy: 0.9326
Epoch 8/20
2403/2403 [==============================] - 76s 31ms/step - loss: 0.1363 - accuracy: 0.9335
Epoch 9/20
2403/2403 [==============================] - 75s 31ms/step - loss: 0.1364 - accuracy: 0.9332
Epoch 10/20
2403/2403 [==============================] - 76s 31m

In [22]:
model = get_model(num_users, num_items, mf_dim=64, layers=[256, 128, 64, 32], reg_layers=[0,0,0,0], reg_mf = [0,0])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
results["MLP_" + str(6) + "_N_" + "lat_" + str(3)] = model.fit(train_data, epochs=20)
predictions["MLP_" + str(6) + "_N_" + "lat_" + str(3)] = model.predict(test_tensor) 

spath = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(6) + "_lat" + str(3) + ".pkl"
with open(spath, 'wb') as file:
    pickle.dump(results["MLP_" + str(6) + "_N_" + "lat_" + str(3)].history, file)
print(f"{spath} -- saved")

path = "/home/jupyter/model_results/" + "mixture_" + "MLP" + str(6) + "_lat" + str(3) + ".npy"
np.save(path, predictions["MLP_" + str(6) + "_N_" + "lat_" + str(3)])
print(f"{path} -- saved")

Epoch 1/20
2403/2403 [==============================] - 90s 37ms/step - loss: 0.4590 - accuracy: 0.7728
Epoch 2/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.3599 - accuracy: 0.8369
Epoch 3/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.2209 - accuracy: 0.9069
Epoch 4/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.1751 - accuracy: 0.9231
Epoch 5/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.1564 - accuracy: 0.9285
Epoch 6/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.1467 - accuracy: 0.9307
Epoch 7/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.1404 - accuracy: 0.9324
Epoch 8/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.1386 - accuracy: 0.9325
Epoch 9/20
2403/2403 [==============================] - 89s 37ms/step - loss: 0.1378 - accuracy: 0.9327
Epoch 10/20
2403/2403 [==============================] - 89s 37m